## 1. Web Scraping

Modify the scripts we used in class to make a program to download both tables present in the  wikipedia page on the Anscombe's Quartet (https://en.wikipedia.org/wiki/Anscombe%27s_quartet). Each table should be saved in its own csv file. **Note:** Your file for the first table should contain the column names, the file for the second table does not need the column names.

In [117]:
import pandas as pd

# read in the tables
wiki_tables = pd.read_html('https://en.wikipedia.org/wiki/Anscombe%27s_quartet')

# name each table
property_table = wiki_tables[0]
anscombe_table = wiki_tables[1]

# write tables to csv
property_table.to_csv('Property_table.csv')
anscombe_table.to_csv('Anscombe_table.csv')

## 2. Pandas and Stats

The Iris dataset is one of the most famous datasets in statistics. Read about it in wikipedia: https://en.wikipedia.org/wiki/Iris_flower_data_set.

Download the dataset from the table in the wikipedia page using beatifulsoup or pandas, create a pandas dataframe containing the dataset (including column names). **Note:** The first column of the table contains only the order of the points in the dataset, it should become the index of your data frame.

In [118]:
import requests
from bs4 import BeautifulSoup

# store webpage
page  = requests.get("https://en.wikipedia.org/wiki/Iris_flower_data_set")

# store webpage data
data = page.text
soup = BeautifulSoup(data,'html5lib')

for table in soup.find_all("table"):
    if table.find('caption'):
        fullTable = []
        for tr in table.find_all('tr'):
            line = []
            if tr.find_all('th'):
                columnNames = []
                for th in tr.find_all('th'):
                    columnNames.append(th.get_text().strip())
            else:
                for td in tr.find_all('td'):
                    line.append(td.get_text().strip())
                fullTable.append(line)

if len(columnNames) == len(fullTable[1]):
    newTable = pd.DataFrame(fullTable, columns = columnNames)
else:
    newTable = pd.DataFrame(fullTable)

# make dataset order the index of the table
newTable.set_index('Dataset order', inplace=True)
newTable

,Sepal length,Sepal width,Petal length,Petal width,Species
Dataset order,,,,,
1,5.1,3.5,1.4,0.2,I. setosa
2,4.9,3.0,1.4,0.2,I. setosa
3,4.7,3.2,1.3,0.2,I. setosa
4,4.6,3.1,1.5,0.2,I. setosa
5,5.0,3.6,1.4,0.3,I. setosa
...,...,...,...,...,...
146,6.7,3.0,5.2,2.3,I. virginica
147,6.3,2.5,5.0,1.9,I. virginica
148,6.5,3.0,5.2,2.0,I. virginica


Your dataframe might have string values in the columns, if so, you need to convert each of the columns that should contain numbers to numeric values (Check the function `pd.to_numeric`).

After converting the columns to numeric use the `desribe()` method to  calculate the average and standard deviation for each variable.

In [119]:
# convert all cols except Species into numric datatypes
for col in newTable.columns:
    if col != 'Species': 
        newTable[col] = pd.to_numeric(newTable[col], errors='coerce')

# descriptive statistics
newTable.describe()

,Sepal length,Sepal width,Petal length,Petal width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.200000
std,0.828066,0.435866,1.765298,0.761401
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


Use the `.groupby()` method to group the data by species and calculate the average and standard deviation for each variable based on the iris species.

In [120]:
# Group by species type and get descriptive stats
grouped_iris = newTable.groupby('Species').describe()

# select the mean and std
iris_mean_std = grouped_iris.loc[:, (slice(None), ['mean', 'std'])] # level based indexing

iris_fin = pd.DataFrame(iris_mean_std)
iris_fin

Sepal length           Sepal width           Petal length  \
                      mean       std        mean       std         mean   
Species                                                                   
I. setosa            5.006  0.352490       3.428  0.379064        1.462   
I. versicolor        5.936  0.516171       2.770  0.313798        4.260   
I. virginica         6.588  0.635880       2.974  0.322497        5.552   

                        Petal width            
                    std        mean       std  
Species                                        
I. setosa      0.173664       0.248  0.105444  
I. versicolor  0.469911       1.326  0.197753  
I. virginica   0.551895       2.026  0.274650

Make scatter plot showing the covariance of the variables. Check plotly's `create_scatterplotmatrix` function from the `figure_factory`. Your graph should look like this:

<img src="iris.png"></img>

In [128]:
import plotly.figure_factory as ff

# scatter plot matrtix
fig = ff.create_scatterplotmatrix(newTable, 
                                  diag='histogram',
                                  index='Species',
                                  colormap_type='cat',
                                  height=800, 
                                  width=800)

fig.update_layout(template='plotly_white',
                  title={'x':0.5}
                  )
fig.show()
